<h1><font color="#113D68" size=6>Tema 4 - Sistemas Multiagentes</font></h1>

<h1><font color="#113D68" size=5>Ampliación de Sistemas Inteligentes (CÓDIGO 71014069)</font></h1>

<h1><font color="#113D68" size=4>3. Comunicación y Plataformas Multiagente</font></h1>

<br><br>
<div style="text-align: right">
<font color="#113D68" size=3>Manuel Castillo-Cara</font><br>
</div>


---

La comunicación es un componente esencial para la **coordinación, cooperación y negociación** en sistemas multiagente (SMA).  
A diferencia de los intercambios informales, los agentes utilizan **lenguajes de comunicación normalizados** para garantizar interoperabilidad y entendimiento semántico.

El estándar más utilizado es **FIPA-ACL (Foundation for Intelligent Physical Agents - Agent Communication Language)**, que define:
- **Performativas (speech acts)**: tipo de intención comunicativa del mensaje.
- **Estructura estándar** del mensaje (remitente, receptor, contenido, intención, protocolo).
- **Protocolos de interacción** que describen el flujo de diálogo (p. ej., *request–reply*, *contract net*, *inform–agree*).

> El objetivo de esta práctica es **simular la comunicación estructurada entre agentes** en Python, y analizar cómo los distintos modos (síncrono/asíncrono, directo/broadcast) afectan al rendimiento y la coherencia de las interacciones.

---

<a id="indice"></a>
# <font color="#004D7F" size=6>Índice</font>

1. [Representación de mensajes FIPA-ACL](#intro)
2. [Simulación básica de mensajería entre agentes](#api)
3. [Ejemplo: Diálogo request–reply](#algos)
4. [Comunicación asíncrona vs síncrona (comparativa)](#obj)
5. [Modelo de Mensaje ACL y agentes mínimos](#single)
6. [Simulador interactivo: colas e intercambio](#multirun)
7. [Interfaz interactiva](#bench)
8. [Conclusiones](#vis2d)


---

<a id="intro"></a>
# <font color="#004D7F" size=6>1. Representación de mensajes FIPA-ACL</font>

Cada mensaje FIPA-ACL contiene:
- **Performativa:** intención comunicativa (`request`, `inform`, `propose`, `agree`, etc.).  
- **Sender/Receiver:** agentes implicados.  
- **Content:** información o acción solicitada.  
- **Conversation ID:** identificador único que agrupa los mensajes de un mismo diálogo.  
- **Timestamp:** marca temporal para trazar la secuencia.

Esto reproduce la semántica de los *speech acts* definida por FIPA (Secciones 20.4–20.5).

In [1]:
# ==== Mensajes FIPA-ACL simplificados ====
from dataclasses import dataclass
from datetime import datetime
import uuid

@dataclass
class ACLMessage:
    performative: str
    sender: str
    receiver: str
    content: str
    conversation_id: str = None
    timestamp: datetime = None

    def __post_init__(self):
        if self.conversation_id is None:
            self.conversation_id = str(uuid.uuid4())[:8]
        if self.timestamp is None:
            self.timestamp = datetime.now()

    def __str__(self):
        return (f"[{self.performative.upper()}] {self.sender} → {self.receiver}: "
                f"{self.content} ({self.conversation_id})")

---

<a id="api"></a>
# <font color="#004D7F" size=6>2. Simulación básica de mensajería entre agentes</font>

En este mini-simulador:
- El objeto `World` actúa como *middleware de mensajería* (similar al contenedor JADE).
- Los agentes mantienen una **cola FIFO de mensajes** (`inbox`).
- Cada envío crea un objeto `ACLMessage`, que viaja de un agente a otro respetando la semántica del lenguaje FIPA-ACL.

> Esto permite modelar diferentes **protocolos de interacción**, desde simples diálogos *request–inform* hasta subastas *contract net*.

In [2]:
# ==== Simulación de agentes con colas de mensajes ====
from collections import deque

class Agent:
    def __init__(self, name):
        self.name = name
        self.inbox = deque()

    def send(self, msg: ACLMessage, world):
        world.route_message(msg)

    def receive(self):
        if self.inbox:
            msg = self.inbox.popleft()
            print(f"{self.name} recibió: {msg}")

class World:
    def __init__(self):
        self.agents = {}

    def register(self, agent):
        self.agents[agent.name] = agent

    def route_message(self, msg: ACLMessage):
        if msg.receiver in self.agents:
            self.agents[msg.receiver].inbox.append(msg)

---

<a id="algos"></a>
# <font color="#004D7F" size=6>3. Ejemplo: Diálogo request–reply</font>

Este patrón de comunicación es uno de los más utilizados:
1. **Request**: un agente solicita información o acción.
2. **Inform/Refuse**: el receptor responde con el resultado o una negativa.

Este flujo refleja el **acto de habla** entre agentes y se usa en la coordinación de servicios, consultas de estado o confirmaciones.

In [3]:
# ==== Ejemplo de comunicación request–reply ====
A = Agent("AgenteA")
B = Agent("AgenteB")
world = World()
world.register(A)
world.register(B)

# Agente A envía una petición
msg1 = ACLMessage("request", sender="AgenteA", receiver="AgenteB",
                  content="¿Cuál es el estado de la tarea T3?")
A.send(msg1, world)

# Agente B procesa y responde
msg2 = ACLMessage("inform", sender="AgenteB", receiver="AgenteA",
                  content="La tarea T3 está completada.",
                  conversation_id=msg1.conversation_id)
B.send(msg2, world)

# Visualización del intercambio
A.receive()
B.receive()

AgenteA recibió: [INFORM] AgenteB → AgenteA: La tarea T3 está completada. (1e6b45a8)
AgenteB recibió: [REQUEST] AgenteA → AgenteB: ¿Cuál es el estado de la tarea T3? (1e6b45a8)


---

<a id="obj"></a>
# <font color="#004D7F" size=6>4. Comunicación asíncrona vs síncrona (comparativa)</font>

- **Síncrona:** el emisor espera una respuesta antes de continuar; garantiza orden pero puede ralentizar la ejecución.  
- **Asíncrona:** los mensajes se procesan en segundo plano, lo que mejora la escalabilidad pero introduce posibles *delays* o *race conditions*.  

> En plataformas reales como **JADE**, este comportamiento se gestiona mediante **comportamientos cíclicos** (`CyclicBehaviour`) y **contenedores de mensajes**, garantizando persistencia y entrega fiable.

In [4]:
import random, time

def simulate_communication(world, agents, mode="async", steps=5):
    for i in range(steps):
        sender, receiver = random.sample(agents, 2)
        msg = ACLMessage("inform", sender=sender.name, receiver=receiver.name,
                         content=f"Mensaje {i} ({mode})")
        sender.send(msg, world)
        if mode == "sync":
            receiver.receive()
        else:
            if random.random() < 0.5:  # recepción aleatoria
                receiver.receive()

agents = [Agent(f"A{i}") for i in range(4)]
world = World()
for ag in agents: world.register(ag)

simulate_communication(world, agents, mode="sync")
simulate_communication(world, agents, mode="async")

A0 recibió: [INFORM] A2 → A0: Mensaje 0 (sync) (56940681)
A3 recibió: [INFORM] A0 → A3: Mensaje 1 (sync) (e5b2c740)
A1 recibió: [INFORM] A0 → A1: Mensaje 2 (sync) (bcbf2306)
A3 recibió: [INFORM] A2 → A3: Mensaje 3 (sync) (126febb9)
A3 recibió: [INFORM] A0 → A3: Mensaje 4 (sync) (fcf5a356)
A3 recibió: [INFORM] A2 → A3: Mensaje 0 (async) (d0834e18)
A3 recibió: [INFORM] A2 → A3: Mensaje 2 (async) (4275a48f)


---

<a id="single"></a>
# <font color="#004D7F" size=6> 5. Modelo de Mensaje ACL y agentes mínimos</font>

**Objetivo.** Explorar cómo los agentes se comunican mediante lenguajes tipo **FIPA-ACL** y cómo distintos **modos de entrega** (síncrono/asíncrono, *unicast*/ *broadcast*) afectan al sistema. El cuaderno incluye salidas **interactivas**: simulación de colas (inboxes) y **cronogramas** de intercambio con métricas de latencia y rendimiento.

**Claves teóricas (resumen):**
- Mensajes ACL con *performativa* (`request`, `inform`, `propose`, `agree`…), `sender`, `receiver`, `content`, `conversation_id`, `timestamp`.
- Patrones de diálogo: *request–reply*, *contract net*, *inform–agree*.
- Entrega **síncrona** vs **asíncrona**: consistencia vs escalabilidad.
- Difusión **unicast** (un receptor) vs **broadcast** (varios receptores).

In [5]:
# ==== Modelo de mensajes y mini-middleware ====
from dataclasses import dataclass
from collections import deque, defaultdict
from datetime import datetime
import uuid, numpy as np
import matplotlib.pyplot as plt

@dataclass
class ACLMessage:
    performative: str
    sender: str
    receiver: str   # 'broadcast' admite lista o el literal 'ALL'
    content: str
    conversation_id: str = None
    t_emit: float = None   # tiempo lógico de emisión

    def __post_init__(self):
        if self.conversation_id is None:
            self.conversation_id = str(uuid.uuid4())[:6]
        if self.t_emit is None:
            self.t_emit = 0.0

class AgentSim:
    def __init__(self, name):
        self.name = name
        self.inbox = deque()
        self.busy_until = 0.0  # para modo síncrono

    def enqueue(self, msg: ACLMessage):
        self.inbox.append(msg)

class WorldSim:
    def __init__(self, n_agents=5, seed=7):
        self.rng = np.random.default_rng(seed)
        self.time = 0.0
        self.agents = {f"A{i}": AgentSim(f"A{i}") for i in range(n_agents)}
        self.events = []  # (t, kind, data) para timeline

    def names(self):
        return list(self.agents.keys())

    def route(self, msg: ACLMessage, delay=0.0, loss_prob=0.0):
        # pérdida de mensajes
        if self.rng.random() < loss_prob:
            self.events.append((msg.t_emit, "drop", (msg.sender, msg.receiver, msg.conversation_id)))
            return

        receivers = self.names() if (msg.receiver == "ALL") else ([msg.receiver] if isinstance(msg.receiver, str) else msg.receiver)
        for r in receivers:
            deliver_t = msg.t_emit + max(0.0, delay + self.rng.normal(0, delay*0.1))
            m2 = ACLMessage(msg.performative, msg.sender, r, msg.content, msg.conversation_id, msg.t_emit)
            self.agents[r].enqueue(m2)
            self.events.append((deliver_t, "deliver", (msg.sender, r, msg.conversation_id)))

    def reset_events(self):
        self.events.clear()

---

<a id="multirun"></a>
# <font color="#004D7F" size=6>6. Simulador interactivo: colas e intercambio</font>

In [7]:
# ==== Motor de simulación con parámetros y generación de métricas ====
from ipywidgets import interact, IntSlider, FloatSlider, Dropdown, Checkbox, HBox, VBox, Layout

def simulate_run(n_agents=6, steps=60, msg_rate=0.6,
                 mode="async", routing="unicast",
                 base_delay=0.02, loss_prob=0.0, seed=7):
    """
    Devuelve:
      - world (con colas procesadas)
      - snapshots: lista de dict {agent: queue_len} por paso
      - timeline: lista de eventos (t, kind, data)
      - msgs_log: lista de (t_emit, sender, receiver, conv)
      - rtts: tiempos ida/vuelta aproximados en 'request–inform' cuando coincide conversation_id
    """
    rng = np.random.default_rng(seed)
    world = WorldSim(n_agents=n_agents, seed=seed)
    names = world.names()
    snapshots = []
    msgs_log = []
    inflight = {}  # conv -> t_emit para estimar RTT
    rtts = []

    for step in range(steps):
        t = step * base_delay

        # generación de mensajes
        for s in names:
            if rng.random() < msg_rate:
                if routing == "unicast":
                    r = rng.choice([x for x in names if x != s])
                else:
                    r = "ALL"
                perf = rng.choice(["inform","request"], p=[0.6,0.4])
                conv = str(uuid.uuid4())[:6]
                msg = ACLMessage(perf, s, r, f"m{step}", conversation_id=conv, t_emit=t)
                world.route(msg, delay=base_delay, loss_prob=loss_prob)
                msgs_log.append((t, s, r, conv, perf))
                if perf == "request":
                    inflight[conv] = t

        # procesamiento "síncrono": si el agente recibe request, no emite nuevo hasta responder
        if mode == "sync":
            for a in names:
                ag = world.agents[a]
                if ag.inbox:
                    m = ag.inbox.popleft()
                    # responder solo a request, con 'inform'
                    if m.performative == "request":
                        reply = ACLMessage("inform", a, m.sender, "ok", conversation_id=m.conversation_id, t_emit=t+base_delay)
                        world.route(reply, delay=base_delay, loss_prob=loss_prob)
                        # RTT
                        if reply.conversation_id in inflight:
                            rtts.append((reply.t_emit - inflight[reply.conversation_id]))
                            inflight.pop(reply.conversation_id, None)
        else:
            # asíncrono: procesa 0..k mensajes al azar (no necesariamente responde)
            k = rng.integers(0, 3)
            for _ in range(k):
                a = rng.choice(names)
                if world.agents[a].inbox:
                    m = world.agents[a].inbox.popleft()
                    if m.performative == "request" and rng.random() < 0.8:
                        reply = ACLMessage("inform", a, m.sender, "ok", conversation_id=m.conversation_id, t_emit=t+base_delay)
                        world.route(reply, delay=base_delay, loss_prob=loss_prob)
                        if reply.conversation_id in inflight:
                            rtts.append((reply.t_emit - inflight[reply.conversation_id]))
                            inflight.pop(reply.conversation_id, None)

        snapshots.append({a: len(world.agents[a].inbox) for a in names})

    world.events.sort(key=lambda x: x[0])
    return world, snapshots, world.events, msgs_log, np.array(rtts)

def plot_queues(snapshots):
    # heatmap de tamaño de cola por agente y paso
    M = np.array([[snap[a] for a in sorted(snap.keys())] for snap in snapshots])
    plt.figure(figsize=(6.6,3.6))
    plt.imshow(M.T, aspect='auto', cmap='viridis')
    plt.yticks(range(M.shape[1]), sorted(snapshots[0].keys()))
    plt.xlabel("Paso"); plt.ylabel("Agente")
    plt.colorbar(label="tamaño inbox")
    plt.title("Evolución de colas (inboxes)")
    plt.tight_layout(); plt.show()

def plot_timeline(events):
    # diagrama de flechas tiempo–agentes
    # y = índice por agente; x = tiempo; color = conversation_id
    if not events:
        print("No hay eventos")
        return
    # codificación de colores por conv
    convs = list({d[2] for _,k,d in events if k=="deliver"})
    cmap = plt.get_cmap("tab20")
    col = {c: cmap(i % 20) for i, c in enumerate(convs)}
    agents = sorted({d[0] for _,k,d in events if k=="deliver"} | {d[1] for _,k,d in events if k=="deliver"})
    ymap = {a:i for i,a in enumerate(agents)}

    plt.figure(figsize=(8.4,4.2))
    for t, k, (s, r, c) in events:
        if k!="deliver": 
            continue
        ys, yr = ymap[s], ymap[r]
        plt.plot([t, t], [ys, yr], color=col[c], alpha=0.7)
        plt.scatter([t], [ys], color=col[c], s=12)
        plt.scatter([t], [yr], color=col[c], s=12)
    plt.yticks(range(len(agents)), agents)
    plt.xlabel("Tiempo lógico"); plt.ylabel("Agente")
    plt.title("Timeline de entregas (color por conversation_id)")
    plt.tight_layout(); plt.show()

def plot_rtt_hist(rtts):
    if len(rtts)==0:
        print("No hay RTTs medidos (faltan conversaciones request→inform).")
        return
    plt.figure(figsize=(5.2,3.2))
    plt.hist(rtts, bins=16)
    plt.xlabel("RTT aproximado"); plt.ylabel("frecuencia")
    plt.title("Distribución de RTT (request→inform)")
    plt.tight_layout(); plt.show()

---

<a id="bench"></a>
# <font color="#004D7F" size=6>7. Interfaz interactiva</font>

**Mini experimento guiado**:

- **Síncrono vs Asíncrono**: fija `msg_rate=0.8` y compara `mode=sync` vs `async`.
  - Esperable: en *sync* habrá menos mensajes pendientes (colas más pequeñas) y **RTT** más concentrados.
  - En *async* aumenta la **concurrencia**, pero se observan colas más pobladas y RTTs más dispersos.
- **Unicast vs Broadcast**: con `routing=broadcast` crece el número de entregas por paso; si `loss_prob` es bajo, la red “se llena” más rápido.
- **Pérdida y retardo**: sube `loss_prob` y `base_delay`; observa cómo empeora el **timeline** (líneas más densas y tardías) y la distribución de **RTT** se desplaza a la derecha.

In [ ]:
# ==== Interfaz: sliders y plots ====
def run_interactive(n_agents, steps, msg_rate, mode, routing, base_delay, loss_prob, seed):
    world, snaps, events, log, rtts = simulate_run(
        n_agents=n_agents, steps=steps, msg_rate=msg_rate,
        mode=mode, routing=routing, base_delay=base_delay,
        loss_prob=loss_prob, seed=seed
    )
    plot_queues(snaps)
    plot_timeline(events)
    plot_rtt_hist(rtts)

_ = interact(
    run_interactive,
    n_agents=IntSlider(6, min=3, max=16, step=1, description="Agentes"),
    steps=IntSlider(60, min=20, max=200, step=10, description="Pasos"),
    msg_rate=FloatSlider(0.6, min=0.1, max=1.0, step=0.05, readout_format=".2f", description="Msg rate"),
    mode=Dropdown(options=["async","sync"], value="async", description="Modo"),
    routing=Dropdown(options=["unicast","broadcast"], value="unicast", description="Ruta"),
    base_delay=FloatSlider(0.02, min=0.0, max=0.2, step=0.01, readout_format=".2f", description="Delay base"),
    loss_prob=FloatSlider(0.0, min=0.0, max=0.3, step=0.02, readout_format=".2f", description="Pérdida"),
    seed=IntSlider(7, min=1, max=999, step=1, description="Seed")
);

interactive(children=(IntSlider(value=6, description='Agentes', max=16, min=3), IntSlider(value=60, descriptio…

---

<a id="vis2d"></a>
# <font color="#004D7F" size=6>8. Conclusiones</font>

Este cuaderno ha mostrado cómo la **comunicación estructurada** entre agentes permite coordinar acciones, intercambiar información y mantener coherencia en entornos distribuidos.

1. **Lenguaje FIPA-ACL y estructura del mensaje**
    - El uso de mensajes con campos explícitos (`performative`, `sender`, `receiver`, `content`, `conversation_id`, `timestamp`) reproduce fielmente la semántica de **FIPA-ACL**.  
    - Esto garantiza **interoperabilidad y claridad intencional**, base de los protocolos de interacción definidos en los estándares FIPA (request–reply, contract-net, inform–agree).

2. **Comunicación síncrona y asíncrona**
    - Los experimentos demostraron cómo los diferentes **modos de entrega** impactan en la dinámica del sistema:
        - En modo **síncrono**, los agentes esperan respuesta antes de continuar, lo que reduce el número de mensajes pendientes (colas pequeñas) y genera **baja latencia** (RTT concentrado).  
        - En modo **asíncrono**, la comunicación es más fluida y escalable, pero produce colas más pobladas y **mayor variabilidad temporal**, reflejando comportamientos realistas en redes distribuidas.
    - El análisis de los *timelines* evidenció que el modo asíncrono favorece la **concurrencia**, mientras que el síncrono preserva la **coherencia secuencial**.

3. Unicast y broadcast
    - El cambio del modo **unicast** al **broadcast** mostró el efecto de la **difusión masiva** de mensajes: aumenta drásticamente la carga de entrega, pero mejora la **propagación de información** entre agentes.  
    - Este comportamiento reproduce el dilema clásico entre **eficiencia local** y **visibilidad global** en sistemas multiagente.

4. **Métricas y cronogramas**
    - Las visualizaciones interactivas de:
        - **colas de mensajes**,  
        - **cronogramas de entrega (timeline)**,  
        - y **distribuciones de RTT**,  
    - permitieron observar en tiempo real cómo la estructura de comunicación y el nivel de pérdida influyen en la **latencia**, **congestión** y **rendimiento colectivo**.  
    - Estos gráficos reflejan las mismas métricas que en plataformas reales de mensajería distribuida.

5. **Síntesis general**
    - La comunicación estructurada constituye el **núcleo funcional de los sistemas multiagente**.  
    - A través de protocolos bien definidos y control del flujo de mensajes, los agentes pueden **coordinarse, cooperar y negociar**, alcanzando comportamientos colectivos coherentes sin necesidad de un control centralizado.  